In [6]:
import sqlite3 as sql 
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
conn = sql.connect('ETL.db')
cursor = conn.cursor()

In [15]:
df = pd.read_sql_query("SELECT * FROM Peso_argentino", conn)
df

,moeda,nome,compra,venda,dataAtualizacao
0,ARS,Peso Argentino,0.0037,0.0037,2026-01-21T21:02:12.209Z


In [5]:
url = 'https://br.dolarapi.com/v1/cotacoes'
response = requests.get(url)            
if response.status_code != 200:
    raise Exception('Erro ao acessar a API')
data = response.json()
data

[{'moeda': 'ARS',
  'nome': 'Peso Argentino',
  'compra': 0.0037,
  'venda': 0.0037,
  'fechoAnterior': 0.0037,
  'dataAtualizacao': '2026-01-21T21:02:12.209Z'},
 {'moeda': 'USD',
  'nome': 'Dólar',
  'compra': 5.311,
  'venda': 5.313,
  'fechoAnterior': 5.3764,
  'dataAtualizacao': '2026-01-20T21:58:36.000Z'},
 {'moeda': 'EUR',
  'nome': 'Euro',
  'compra': 6.21126,
  'venda': 6.22676,
  'fechoAnterior': 6.30556,
  'dataAtualizacao': '2026-01-21T21:03:12.000Z'},
 {'moeda': 'CLP',
  'nome': 'Peso Chileno',
  'compra': 0.006075,
  'venda': 0.006077,
  'fechoAnterior': 0.006067,
  'dataAtualizacao': '2026-01-21T21:00:36.338Z'},
 {'moeda': 'UYU',
  'nome': 'Peso Uruguayo',
  'compra': 0.1387,
  'venda': 0.1388,
  'fechoAnterior': 0.1399,
  'dataAtualizacao': '2026-01-21T21:00:36.338Z'}]

In [12]:
cursor.execute('''
               
       create table if not exists Peso_argentino (
               

                moeda text not null,
                nome text not null,
                compra real not null,
                venda real not null,
                dataAtualizacao text not null

               
               )     
        ''')

In [11]:
cursor.execute('drop table Peso_argentino')

In [13]:
cotacao_peso = []
for cotacao in data:
    if cotacao ['moeda'] == 'ARS':
        cotacao_peso.append({

            'moeda': cotacao['moeda'],
            'nome': cotacao['nome'],   
            'compra': cotacao['compra'],
            'venda': cotacao['venda'],
            'dataAtualizacao': cotacao['dataAtualizacao']



        })

In [14]:
for cotacao in cotacao_peso:
    cursor.execute('''INSERT INTO Peso_argentino (moeda, nome, dataAtualizacao, compra, venda) 
                      VALUES (?, ?, ?, ?, ?)''', 
                   (cotacao['moeda'], cotacao['nome'], cotacao['dataAtualizacao'], cotacao['compra'], cotacao['venda']))